<a href="https://colab.research.google.com/github/chetan-parthiban/Conceptors/blob/master/BERT_Bias_Improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Setup

In [0]:
!pip install pytorch-pretrained-bert
!pip install mlxtend

     |████████████████████████████████| 133kB 9.5MB/s 


In [0]:
import torch
import torch.nn.functional as F
from torch import nn, optim
import numpy as np
from numpy.linalg import inv
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline
dtype = torch.long
device = torch.device('cuda:0')

from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import logging
logging.basicConfig(level=logging.INFO)

torch.cuda.get_device_name(0)

'Tesla T4'

In [0]:
# Gender word lists
!git clone https://github.com/uclanlp/gn_glove
!git clone https://github.com/uclanlp/corefBias
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
    
# our code for debiasing -- also includes word lists    
!git clone https://github.com/jsedoc/ConceptorDebias

# Get Winobias word lists
winoWordsPath = './' + 'corefBias/WinoBias/wino/extra_gendered_words.txt'
male_vino_extra = []
female_vino_extra = []
with open(winoWordsPath, "r+") as f_in:
    for line in f_in:
        male_vino_extra.append(line.split('\t')[0])
        female_vino_extra.append(line.strip().split('\t')[1])

# Get CMU word lists
cmuMaleWordPath = './' + 'male.txt'
male_cmu = []
with open(cmuMaleWordPath, "r+") as f_in:
  for line in f_in:
    w = line.strip()
    if len(w)>0 and w[0] != '#':
      male_cmu.append(w)
cmuFemaleWordPath = './' + 'female.txt'
female_cmu = []
with open(cmuFemaleWordPath, "r+") as f_in:
  for line in f_in:
    w = line.strip()
    if len(w)>0 and w[0] != '#':
      female_cmu.append(w)
    
# Get gnGlove word lists
gnGloveFemaleWordPath = './' + 'gn_glove/wordlist/female_word_file.txt'
female_gnGlove = []
with open(gnGloveFemaleWordPath, "r+") as f_in:
    for line in f_in:
        female_gnGlove.append(line.strip())
gnGloveMaleWordPath = './' + 'gn_glove/wordlist/male_word_file.txt'
male_gnGlove = []
with open(gnGloveMaleWordPath, "r+") as f_in:
    for line in f_in:
        male_gnGlove.append(line.strip())
    
# Get WEAT lists and conceptor functionality
from ConceptorDebias.Conceptors.conceptor_fxns import *
from ConceptorDebias.lists import WEAT_lists
WEATLists = WEAT_lists.WEATLists()

Cloning into 'gn_glove'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 162 (delta 18), reused 25 (delta 9), pack-reused 114
Receiving objects: 100% (162/162), 73.36 KiB | 790.00 KiB/s, done.
Resolving deltas: 100% (64/64), done.
Cloning into 'corefBias'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 471 (delta 3), reused 0 (delta 0), pack-reused 457
Receiving objects: 100% (471/471), 84.18 MiB | 31.00 MiB/s, done.
Resolving deltas: 100% (273/273), done.
--2019-06-07 17:14:31--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35751 (35K) [text/plain]
Saving to: ‘female.txt’

fe

In [0]:
gender_list_pronouns = WEATLists.W_7_Male_terms + WEATLists.W_7_Female_terms + WEATLists.W_8_Male_terms + WEATLists.W_8_Female_terms
gender_list_pronouns = list(set(gender_list_pronouns))

pronouns_male = WEATLists.W_7_Male_terms + WEATLists.W_8_Male_terms
pronouns_male = list(set(pronouns_male))
pronouns_female = WEATLists.W_8_Female_terms + WEATLists.W_8_Female_terms
pronouns_female = list(set(pronouns_female))


gender_list_extended = male_vino_extra + female_vino_extra + male_gnGlove + female_gnGlove
gender_list_extended = list(set(gender_list_extended))

extended_male = male_vino_extra + male_gnGlove
extended_male = list(set(extended_male))
extended_female = female_vino_extra + female_gnGlove
extended_female = list(set(extended_female))

gender_list_propernouns = male_cmu + female_cmu
gender_list_propernouns = list(set(gender_list_propernouns))

propernouns_male = list(set(male_cmu))
propernouns_female = list(set(female_cmu))

gender_list_all = gender_list_pronouns + gender_list_extended + gender_list_propernouns
gender_list_all = list(set(gender_list_all))

all_male = list(set(pronouns_male + extended_male + propernouns_male))
all_female = list(set(pronouns_female + extended_female + propernouns_female))

career = list(set(WEATLists.W_6_Career))
family = list(set(WEATLists.W_6_Family))

## Initialization

In [0]:
bert_model = 'bert-base-uncased'

## Get BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(bert_model)
model = BertForMaskedLM.from_pretrained(bert_model)
model.eval()
model.to('cuda')

In [0]:
sentence_template = '[CLS] [MASK] likes [MASK] [SEP]'

target1 = ['male', 'man', 'boy', 'brother', 'he', 'him', 'his', 'son']
target2 = ['female', 'woman', 'girl', 'sister', 'she', 'her', 'hers', 'daughter']
attribute1 = ['math', 'algebra', 'geometry', 'calculus', 'equations', 'computation', 'numbers', 'addition']
attribute2 = ['poetry', 'art', 'dance', 'literature', 'novel', 'symphony', 'drama', 'sculpture']
t_position = 1
a_position = 3

print('Target 1: ', target1)
print('Target 2: ', target2)
print('Attribute 1: ', attribute1)
print('Attribute2 2: ', attribute2)

template_token = tokenizer.tokenize(sentence_template)
segments_ids = np.zeros(len(template_token))

Target 1:  ['male', 'man', 'boy', 'brother', 'he', 'him', 'his', 'son']
Target 2:  ['female', 'woman', 'girl', 'sister', 'she', 'her', 'hers', 'daughter']
Attribute 1:  ['math', 'algebra', 'geometry', 'calculus', 'equations', 'computation', 'numbers', 'addition']
Attribute2 2:  ['poetry', 'art', 'dance', 'literature', 'novel', 'symphony', 'drama', 'sculpture']


In [0]:
def sentence_from_template(attribute, tokenized, pos):
    
    sentences = []

    for atr in attribute:
        temp = tokenized.copy()
        temp[pos] = atr
        sentences.append(temp)
                
    return sentences

a1 = sentence_from_template(attribute1, template_token, a_position)
a2 = sentence_from_template(attribute2, template_token, a_position)

In [0]:
def sentences_to_ids(sentences, tokenizer, pos):  
    ids = []
    attribute_ids = []
    
    for sentence in sentences:
        sentence_ids = tokenizer.convert_tokens_to_ids(sentence)
        sentence_attribute = tokenizer.convert_tokens_to_ids([sentence[pos]])
        
        ids.append(sentence_ids)
        attribute_ids.append(sentence_attribute)
        
    return ids, attribute_ids

target1_id = tokenizer.convert_tokens_to_ids(target1)
target2_id = tokenizer.convert_tokens_to_ids(target2)

a1_id, attribute_ids = sentences_to_ids(a1, tokenizer, a_position)
a2_id, attribute_ids = sentences_to_ids(a2, tokenizer, a_position)

masked = [tokenizer.convert_tokens_to_ids(template_token)]

In [0]:
a1_tensor = torch.tensor(a1_id, device = device, dtype = dtype)
a2_tensor = torch.tensor(a2_id, device = device, dtype = dtype)
masked_tensor = torch.tensor(masked, device = device, dtype = dtype)
segments_tensor = torch.tensor([segments_ids], device = device, dtype = dtype)

In [0]:
with torch.no_grad():
    p1 = model(a1_tensor, segments_tensor)
    p2 = model(a2_tensor, segments_tensor)
    p_init = model(masked_tensor, segments_tensor)
    
s = nn.Softmax(dim = 1)
p1 = s(p1[:,t_position])
p2 = s(p2[:,t_position])
p_init = s(p_init[:,t_position])

In [0]:
p_targets1 = p1[:,target1_id]
p_targets2 = p2[:,target1_id]
p_targets_init1 = p_init[:,target1_id]

p1_ratio = p_targets1 / p_targets_init1
p2_ratio = p_targets2 / p_targets_init1


logp1t1 = torch.log(p1_ratio)
logp2t1 = torch.log(p2_ratio)

meanlog1 = torch.mean(logp1t1, dim = 1)
meanlog2 = torch.mean(logp2t1, dim = 1)

s1 = torch.sum(meanlog1)
s2 = torch.sum(meanlog2)

diff1 = s1 - s2

p_targets1 = p1[:,target2_id]
p_targets2 = p2[:,target2_id]
p_targets_init1 = p_init[:,target2_id]

p1_ratio = p_targets1 / p_targets_init1
p2_ratio = p_targets2 / p_targets_init1

logp1t2 = torch.log(p1_ratio)
logp2t2 = torch.log(p2_ratio)

meanlog1 = torch.mean(logp1t2, dim = 1)
meanlog2 = torch.mean(logp2t2, dim = 1)

s1 = torch.sum(meanlog1)
s2 = torch.sum(meanlog2)

diff2 = s1 - s2

In [0]:
def permutation_test(t1a1,t1a2,t2a1,t2a2, sample_size = 10000):
    
    statistic = (torch.sum(torch.mean(t1a1, dim = 1)) - torch.sum(torch.mean(t1a2, dim = 1))) \
            - (torch.sum(torch.mean(t2a1, dim = 1)) - torch.sum(torch.mean(t2a2, dim = 1)))
    data1 = torch.cat((logp1t1, logp1t2), dim = 1)
    data2 = torch.cat((logp2t1, logp2t2), dim = 1)
    sz = int(data1.size()[0]/2)

    results = []
    for _ in range(sample_size):
        
        perm = np.random.permutation(sz*2)
        
        d11 = data1[perm[0:sz]]
        d12 = data1[perm[sz:]]
        
        d21 = data2[perm[0:sz]]
        d22 = data2[perm[sz:]]
        
        t_stat = (torch.sum(torch.mean(d11, dim = 1)) - torch.sum(torch.mean(d21, dim = 1))) \
            - (torch.sum(torch.mean(d21, dim = 1)) - torch.sum(torch.mean(d22, dim = 1)))
        
        results.append(t_stat)
        
    results2 = [stat for stat in results if stat > statistic]
    
    p_value = len(results2)/sample_size
    std = torch.std(torch.mean(torch.cat((logp1t1 - logp2t1, logp1t2 - logp2t2),1)))
    effect_size = (torch.mean(torch.mean(t1a1, dim = 1)) - torch.mean(torch.mean(t1a2, dim = 1))) \
            - (torch.sum(torch.mean(t2a1, dim = 1)) - torch.mean(torch.mean(t2a2, dim = 1)))
    
    return p_value, effect_size.item()

permutation_test(logp1t1,logp2t1,logp1t2,logp2t2, sample_size = 1000)

(0.436, 7.504287242889404)